In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from meteostat import Point, Daily
from meteostat import Hourly
import holidays
from datetime import datetime, timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def filter_holidays(holidays, start_date, end_date):
    start_ts = pd.Timestamp(start_date)
    end_ts = pd.Timestamp(end_date)

    return {
        date_key: holiday_name
        for date_key, holiday_name in holidays.items()
        if start_ts <= pd.Timestamp(date_key) <= end_ts
    }

def mark_special_days(df, holidays_df):
    # Create time mask for 7am to 1pm
    market_time = (df['timestamp'].dt.hour >= 7) & (df['timestamp'].dt.hour < 13)
    # Create day mask for Wednesdays and Saturdays
    market_day = df['timestamp'].dt.day_name().isin(['Wednesday', 'Saturday'])

    # Create column for Wednesdays and Saturdays
    df['market_day'] = (market_day & market_time).astype(int)

   # Create a temporary series for holidays that fall on Thursday
    #thursday_holidays = (df['timestamp'].dt.date.isin(holidays_df.date_holiday)) & (df['timestamp'].dt.day_name() == 'Thursday')
    holidays_df['date'] = pd.to_datetime(holidays_df['date_holiday'])

    thursday_holidays = holidays_df[holidays_df['date'].dt.day_name() == 'Thursday']
    # Mark Fridays that follow a Thursday holiday
    df['special_friday'] = ((df['timestamp']-timedelta(days=1)).dt.date.isin(thursday_holidays['date'].dt.date) & (df['timestamp'].dt.day_name() == 'Friday')).astype(int)
    #df['special_friday'] = thursday_holidays.shift(-1) & (df['timestamp'].dt.day_name() == 'Friday')
    #df['special_friday'] = df['special_friday'].astype(int)

    return df

data = pd.read_csv('./data/stadtgarage.csv')
events_df = pd.read_excel('./data/events.xlsx')
events_df['Date'] = pd.to_datetime(events_df['Date'])
df = pd.DataFrame(data)

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')
start_date = pd.to_datetime('2023-11-30')
end_date = pd.to_datetime('2024-11-12')
df = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)]
df = df[df['measurement_name'] == 'utilization']

austria_holidays = holidays.country_holidays('AT', subdiv='V', years=range(df['timestamp'].min().year, df['timestamp'].max().year+1))
filtered_holidays = filter_holidays(austria_holidays, start_date, end_date)
holidays_df = pd.DataFrame({'date_holiday': list(filtered_holidays.keys()), 'holiday': 'holiday'})
holidays_df = holidays_df.sort_values('date_holiday')
# Define the coordinates for Dornbirn, Austria
point = Point(47.4145, 9.7381)

weather_data_hourly = Hourly(point, start_date, end_date)
weather_data_hourly = weather_data_hourly.fetch()
weather_data_daily = Daily(point, start_date, end_date)
weather_data_daily = weather_data_daily.fetch()


weather_daily_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wspd', 'wpgt', 'pres']
# Reset index of weather data to make 'time' a regular column
weather_daily = weather_data_daily[weather_daily_cols].reset_index()
weather_hourly_cols = ['temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'wpgt', 'pres', 'tsun']
weather_hourly = weather_data_hourly[weather_hourly_cols].reset_index()

# Round your original df timestamps to the nearest hour for matching
df['hour_timestamp'] = df['timestamp'].dt.floor('h')
df['hour_timestamp'] = pd.to_datetime(df['hour_timestamp'])
# Create a date column for merging
df['date'] = df['timestamp'].dt.date
df['date_holiday'] = df['timestamp'].dt.date
df['date_event'] = df['timestamp'].dt.date
df['date_event'] = pd.to_datetime(df['date_event'])

weather_daily['time'] = weather_daily['time'].dt.date
# Merge using broadcast join
df = pd.merge_ordered(df, holidays_df, fill_method="ffill", left_by="date_holiday")
df = df.merge(weather_daily, left_on='date', right_on='time', how='left')
df = df.merge(weather_hourly, left_on='hour_timestamp',right_on='time',how='left')

df = df.merge(events_df, left_on='date_event', right_on='Date', how='left')
df = mark_special_days(df, holidays_df)
# Clean up if needed
df = df.drop('sensor_id', axis=1)
df = df.drop('date_holiday', axis=1)
df = df.drop('date_event', axis=1)
df = df.drop('time_x', axis=1)
df = df.drop('time_y', axis=1)
df = df.drop('Date', axis=1)
df = df.drop('date', axis=1)
df = df.drop('measurement_name', axis=1)
df_hour = df.groupby('hour_timestamp')['value'].mean().reset_index()
df = df.merge(df_hour, left_on='hour_timestamp', right_on='hour_timestamp', how='left')

df = df.drop('timestamp', axis=1)
df = df.drop('value_x', axis=1)

df = df.drop_duplicates(subset=['hour_timestamp'], keep='first')

# Einzelne oder mehrere Spalten umbenennen
df = df.rename(columns={'hour_timestamp': 'timestamp', 'value_y': 'value'})


print(df)

df.to_csv('data/out_hourly.csv', index=False)

                timestamp holiday  tavg  tmin  tmax  prcp_x  snow  wspd_x  \
0     2023-11-30 00:00:00     NaN   0.8  -1.4   3.9    16.4   0.0     3.4   
7     2023-11-30 01:00:00     NaN   0.8  -1.4   3.9    16.4   0.0     3.4   
14    2023-11-30 02:00:00     NaN   0.8  -1.4   3.9    16.4   0.0     3.4   
21    2023-11-30 03:00:00     NaN   0.8  -1.4   3.9    16.4   0.0     3.4   
29    2023-11-30 04:00:00     NaN   0.8  -1.4   3.9    16.4   0.0     3.4   
...                   ...     ...   ...   ...   ...     ...   ...     ...   
58019 2024-11-11 11:00:00     NaN   6.8   5.7   7.7     4.6   0.0     3.1   
58026 2024-11-11 12:00:00     NaN   6.8   5.7   7.7     4.6   0.0     3.1   
58033 2024-11-11 13:00:00     NaN   6.8   5.7   7.7     4.6   0.0     3.1   
58041 2024-11-11 14:00:00     NaN   6.8   5.7   7.7     4.6   0.0     3.1   
58048 2024-11-11 15:00:00     NaN   6.8   5.7   7.7     4.6   0.0     3.1   

       wpgt_x  pres_x  ...  rhum  prcp_y  wspd_y  wpgt_y  pres_y  tsun  \
0